# Эксперименты с books доменом

In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../')

In [2]:
from pathlib import Path
from tqdm import tqdm
import pickle
from time import time
from tqdm import tqdm
import pandas as pd
import numpy as np

import mlflow

from src.models.BaseModel import TopRecommender
from src.models.ItemBasedRecommenders import CosineDistanceRecommender
from src.models.MatrixFactorizationRecommenders import ALSBasedRecommender, LightFMBasedRecommender

from src.metrics import average_single_precision

C:\Users\trybi\anaconda3\envs\IndexHockey\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn('LightFM was compiled without OpenMP support. '


In [3]:
# Настраиваем MLFlow (Использую его потому что знаком с ним:))
mlflow.set_tracking_uri((Path.cwd() / '../src/models/mlflow_tracking/mlruns'))
mlflow.set_experiment('Books Domain')

<Experiment: artifact_location='file:///C:/Users/trybi/PycharmProjects/MFDP-RecSys/src/models/mlflow_tracking/mlruns/1', creation_time=1681388593965, experiment_id='1', last_update_time=1681388593965, lifecycle_stage='active', name='Books Domain', tags={'mlflow.note.content': 'Эксперименты с рекомендацией книг для пользователей'}>

In [4]:
data_path = Path('../data')

In [5]:
df_books_rating = pd.read_parquet(data_path / 'df_books_rating.parquet')
books_df = pd.read_parquet(data_path/'books_bd.parquet')

In [6]:
df_books_rating['Book-Rating'] = df_books_rating['Book-Rating'].replace(0, np.nan)

In [7]:
# Заполнение 0 (неявная оценка - прочитал, но не оценил) - заполняем средним по датасету
df_books_rating['Book-Rating'] = df_books_rating['Book-Rating'].fillna(df_books_rating['Book-Rating'].mean())

In [8]:
df_books_rating = df_books_rating.reset_index().drop('index', axis=1)

In [9]:
# Отложим 1000 наблюдений в качестве тестовой выборки. На них будем считать метрику MAP@10.
# (не на 20 потому что будем счиаить, что еще 10 уйдут на другой домен). 
# Немного наблюдений в качестве теста, потому что кажется, что дело это не быстрое. 

test_samples = np.random.choice(range(len(df_books_rating)), size = 1000, replace=False)

df_train = df_books_rating.drop(test_samples)
df_test = df_books_rating.iloc[test_samples]

In [10]:
df_train

,User-ID,ISBN,Book-Rating
0,276746,b0425115801,7.719876
1,276746,b0449006522,7.719876
2,276746,b0553561618,7.719876
3,276746,b055356451X,7.719876
4,276747,b0060976845,9.000000
...,...,...,...
433818,276704,b0441007813,7.719876
433819,276704,b0446353957,7.719876
433820,276704,b0446605409,7.719876
433821,276704,b0743211383,7.000000


In [11]:
df_test

,User-ID,ISBN,Book-Rating
126502,78973,b0451186362,8.000000
208971,132199,b0312199430,8.000000
3821,1548,b0553258540,9.000000
56641,35006,b0440234743,7.000000
85731,52853,b0440222656,8.000000
...,...,...,...
391783,248479,b0060987324,8.000000
33172,20201,b081257639X,9.000000
38470,23933,b0385319037,7.000000
70324,41455,b0892965258,7.719876


In [12]:
# Соберем словарь {user: [items]}, потому что так быстрее будет работать поиск истории по юзерам
train_db = {user: items.item().split(', ') for user, items in (df_train
                                                               .groupby('User-ID')
                                                               .agg({'ISBN': lambda x: ', '.join(x)})
                                                               .iterrows())}

In [13]:
# Посмотрим максимальное количество лайкнутых книг. Спойдер - очень много
max(len(i) for i in train_db.values())

4473

In [14]:
# Соберем список пар (user: item) из test, чтобы по нему итерироваться
test_data = df_test[['User-ID', 'ISBN']].values

In [15]:
model_path = Path('../src/models/models_storage')

In [16]:
# Функция, которая будет считать эксперименты 
def experiment(run_name, model, model_params, recommend_params, my_favorites, k=10):
    with mlflow.start_run(run_name=run_name):
        mlflow.log_params(model_params)
        
        time_start = time()
        model.fit(interaction_data=df_train.rename({'User-ID': 'user', 'ISBN': 'item', 'Book-Rating': 'rating'}, axis=1),
                  **model_params)
        mlflow.log_metric(f'fit_time', time()-time_start)
        
        maps = []
        times = []
        for new_user, new_item in tqdm(test_data):
            user_favorites = train_db.get(new_user, [])
            time_start = time()
            recs = model.recommend(user_favorites, k, **recommend_params)
            times.append(time()-time_start)
            maps.append(average_single_precision(new_item, recs))
        
        
        mlflow.log_metric(f'MAP_on_{k}', sum(maps)/len(maps))
        mlflow.log_metric(f'time_to_rec', sum(times)/len(times))
        
        my_recs = model.recommend(my_favorites, k, **recommend_params)
        
        
        mlflow.log_text(' \n '.join(i + ' - ' + j for k in my_recs for i, j in books_df.query('ISBN==@k')[['Book-Title', 
                                                                                                        'Book-Author']].values),
                        'my_recs.txt')

        with open(model_path / f'{run_name}_model.pickle', 'wb') as f:
            pickle.dump(model, f)

        mlflow.log_artifact(model_path / f'{run_name}_model.pickle')

In [17]:
my_favorites = ['b1853262005', 'b0451526341', 'b0802130119', 'b0805062971', 'b0385333846']

In [18]:
# Интересно, что посоветует мне каждый из алгоритмов. Быстро накидал любимых книг, которые вспомнил сразу
books_df[books_df['ISBN'].isin(my_favorites)]

,Book-Title,Book-Author,ISBN,Year-Of-Publication,Image-URL-M
14991,Animal Farm,George Orwell,b0451526341,1956,http://images.amazon.com/images/P/0451526341.0...
42652,Crime and Punishment (Wordsworth Classics),Fyodor Dostoevsky,b1853262005,1997,http://images.amazon.com/images/P/1853262005.0...
68582,Fight Club,Chuck Palahniuk,b0805062971,1996,http://images.amazon.com/images/P/0805062971.0...
167962,Slaughterhouse-Five,KURT VONNEGUT,b0385333846,1999,http://images.amazon.com/images/P/0385333846.0...
204880,The Master and Margarita,Mikhail Bulgakov,b0802130119,1987,http://images.amazon.com/images/P/0802130119.0...


In [425]:
experiment('Top count recommends', TopRecommender(), {'n_to_save':5000, 'metric':'count'}, {'mode':'deterministic'},
           my_favorites, k=10)

100%|████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 1016.27it/s]


In [428]:
experiment('Top Rating recommends', TopRecommender(), {'n_to_save':5000, 'metric':'mean'}, {'mode':'deterministic'}, 
           my_favorites, k=10)

100%|████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 1000.04it/s]


In [430]:
experiment('Random count recommends', TopRecommender(), {'n_to_save':5000, 'metric':'count'}, {'mode':'probabilistic'},
           my_favorites, k=10)

100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:03<00:00, 309.01it/s]


In [431]:
experiment('CosineDistanceRecommender', CosineDistanceRecommender(), {}, {'mode':'deterministic'},
           my_favorites, k=10)

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [03:37<00:00,  4.61it/s]


In [432]:
experiment('CosineDistanceRecommender + random sample recs', CosineDistanceRecommender(), {}, {'mode':'probabilistic'},
           my_favorites, k=10)

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [03:31<00:00,  4.73it/s]


In [440]:
experiment('ALS Recommender', ALSBasedRecommender(), {}, {},
           my_favorites, k=10)

  0%|          | 0/15 [00:00<?, ?it/s]

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [01:28<00:00, 11.32it/s]


In [442]:
experiment('ALS Recommender', ALSBasedRecommender(), {'factors': 50, 'iterations': 30}, {},
           my_favorites, k=10)

  0%|          | 0/30 [00:00<?, ?it/s]

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [01:19<00:00, 12.53it/s]


In [443]:
experiment('ALS Recommender', ALSBasedRecommender(), {'factors': 200, 'iterations': 30}, {},
           my_favorites, k=10)

  0%|          | 0/30 [00:00<?, ?it/s]

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [01:54<00:00,  8.76it/s]


In [444]:
experiment('LightFM Recommender', LightFMBasedRecommender(), {}, {'epochs': 1, 'verbose': False},
           my_favorites, k=10)

Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19


  0%|                                                                                         | 0/1000 [00:00<?, ?it/s]C:\Users\trybi\anaconda3\envs\IndexHockey\lib\site-packages\scipy\sparse\_index.py:116: SparseEfficiencyWarning: Changing the sparsity structure of a csc_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray_sparse(i, j, x)
100%|████████████████████████████████████████████████████████████████████████████| 1000/1000 [2:05:54<00:00,  7.55s/it]


### Из-за того, что немного изменил модел пришлось переучить лучшие модели

In [19]:
experiment('ALS Recommender new', ALSBasedRecommender(), {}, {},
           my_favorites, k=10)

C:\Users\trybi\anaconda3\envs\IndexHockey\lib\site-packages\implicit\utils.py:34: UserWarning: Intel MKL BLAS detected. Its highly recommend to set the environment variable 'export MKL_NUM_THREADS=1' to disable its internal multithreading
  "Intel MKL BLAS detected. Its highly recommend to set the environment "


  0%|          | 0/15 [00:00<?, ?it/s]

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [01:37<00:00, 10.24it/s]


In [21]:
experiment('ALS Recommender new', ALSBasedRecommender(), {'factors': 50, 'iterations': 30}, {},
           my_favorites, k=10)

  0%|          | 0/30 [00:00<?, ?it/s]

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [01:48<00:00,  9.20it/s]
